**Image Preprocessing**

In [ ]:
import cv2,os
import numpy as np

In [ ]:
data_path='data'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)

In [ ]:
{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]

In [ ]:
print(len(os.listdir('data/with_mask')))
print(len(os.listdir('data/without_mask')))

In [ ]:
img_size=64
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)

    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        resized=cv2.resize(gray,(img_size,img_size))
        data.append(resized)
        target.append(label_dict[category])

In [ ]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))

In [ ]:
np.save('data3',data)
np.save('target3',target)

**Model Training**

In [ ]:
import cv2,os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D


data=np.load('data3.npy')
target=np.load('target3.npy')


In [ ]:
model=Sequential()

model.add(Conv2D(32,(3,3),input_shape=data.shape[1:]))
model.add(Conv2D(32,(3,3),input_shape=data.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Conv2D(32,(3,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Conv2D(32,(3,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(30, activation='relu'))
model.add(Dropout(0.03))

model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.2)
print(train_data.shape)
print(test_data.shape)
history = model.fit(train_data, train_target,epochs=30,
                              validation_data=(test_data, test_target),
                              )

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'],'r',label='training loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()
plt.plot(history.history['accuracy'],'r',label='training accuracy')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()
print(model.evaluate(test_data,test_target))
model.save("t4.model",save_format="h5")
p=[]
predictions = model.predict(test_data)
for i in predictions:
    if i>0.5:
        p.append(1)
    else:
        p.append(0)
p=np.array(p)
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(accuracy_score(test_target,p))
print(classification_report(test_target,p))
print(confusion_matrix(test_target,p))

**Training**

In [ ]:
import cv2,os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D

data=np.load('data3.npy')
target=np.load('target3.npy')

In [ ]:
model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=data.shape[1:]))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.2)
print(train_data.shape)
print(test_data.shape)
history = model.fit(train_data, train_target,epochs=30,
                              validation_data=(test_data, test_target),
                              )
print(model.evaluate(test_data,test_target))
model.save("t5.model",save_format="h5")
p=[]
predictions = model.predict(test_data)
for i in predictions:
    if i>0.5:
        p.append(1)
    else:
        p.append(0)
p=np.array(p)
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(accuracy_score(test_target,p))
print(classification_report(test_target,p))
print(confusion_matrix(test_target,p))

**Image Detection**

In [ ]:
from keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
model = load_model('t5.model')
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
frame = cv2.imread("")
frame.shape
gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
gray.shape
faces=face_clsfr.detectMultiScale(gray)
while True:
    for x,y,w,h in faces:
        cv2.rectangle(gray,(x,y),(x+w,y+h),(255,0,255),2)
    cv2.imshow('image',gray)
    if cv2.waitKey(1)==27:
        break
cv2.destroyAllWindows()

plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
for x,y,w,h in faces:
    face_img=gray[y:y+h,x:x+w]
    resized=cv2.resize(face_img,(64,64))
    normalized=resized/255.0
    reshaped=np.reshape(normalized,(1,64,64,1))
    result=model.predict(reshaped)
    if result>0.5:
        label=1
        print("Mask")
    else:
        label=0
        print("No mask")
print(label)



In [ ]:
frame = cv2.imread("")
frame.shape
gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
gray.shape
faces=face_clsfr.detectMultiScale(gray)
while True:
    for x,y,w,h in faces:
        cv2.rectangle(gray,(x,y),(x+w,y+h),(255,0,255),2)
    cv2.imshow('image',gray)
    if cv2.waitKey(1)==27:
            break
cv2.destroyAllWindows()
plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
for x,y,w,h in faces:
    face_img=gray[y:y+h,x:x+w]
    resized=cv2.resize(face_img,(64,64))
    normalized=resized/255.0
    reshaped=np.reshape(normalized,(1,64,64,1))
    result=model.predict(reshaped)
    if result>0.5:
        label=1
        print("Mask")
    else:
        label=0
        print("No mask")

print(label)

Email Detection

In [ ]:
from keras.models import load_model
import cv2
import numpy as np
import smtplib
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email import encoders

In [ ]:
model = load_model('t5.model')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

labels_dict={1:'MASK',0:'NO MASK'}
color_dict={1:(0,255,0),0:(0,0,255)}

In [ ]:
source=cv2.VideoCapture(0)
count = 0
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:

        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(64,64))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,64,64,1))
        result=model.predict(reshaped)
        if result>0.5:
            label=1
        else:
            label=0
            if count<10:
                cv2.imwrite("frame%d.png" % count,img)
            gmail_user = ""
            gmail_pwd =  ""
            to = ""
            subject = "Report"
            text = "Picture report"
            msg = MIMEMultipart()
            msg['From'] = gmail_user
            msg['To'] = to
            msg['Subject'] = subject
            msg.attach(MIMEText(text))
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(open("frame%d.png" % count, 'rb').read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition','attachment; filename="%s"' % os.path.basename("frame%d.png" % count))
            msg.attach(part)
            mailServer = smtplib.SMTP("smtp.gmail.com", 587)
            mailServer.starttls()
            mailServer.login(gmail_user, gmail_pwd)
            mailServer.sendmail(gmail_user, to, msg.as_string())
            print("mail send")
            mailServer.close()
            count+=1

        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label],(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)



    cv2.imshow('LIVE',img)
    key=cv2.waitKey(2)
    if(key==27):
        break

cv2.destroyAllWindows()
source.release()


In [ ]:
cv2.destroyAllWindows()
source.release()

**Video Detection**

In [ ]:
from keras.models import load_model
import cv2
import numpy as np
model = load_model('t5.model')

In [ ]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
labels_dict={1:'MASK',0:'NO MASK'}
color_dict={1:(0,255,0),0:(0,0,255)}
source=cv2.VideoCapture(0)
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:

        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(64,64))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,64,64,1))
        result=model.predict(reshaped)
        if result>0.5:
            label=1
        else:
            label=0

        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label],(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)


    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)

    if(key==27):
        break

cv2.destroyAllWindows()
source.release()